In [ ]:
!pip install git+https://github.com/openai/CLIP.git


In [ ]:

!pip install numpy==1.26.4

In [ ]:
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1



In [ ]:
!pip install matplotlib

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import clip

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
!pip install pinecone


In [ ]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec 
load_dotenv()



import uuid

image_paths = ['./image_downloads/image_1.png','./image_downloads/image_16.png','./image_downloads/image_31.png','./image_downloads/image_46.png']
new_image_path = './image_downloads/image_46.png'  
 
def get_feature_for_new_image(new_image_path):
    image = preprocess(Image.open(new_image_path).convert("RGB"))
    image = image.unsqueeze(0).to(device)
    with torch.no_grad():
        image_feature = model.encode_image(image).float()
    print(f"single_image : {image_feature.squeeze(0).shape}")     
    return image_feature.squeeze(0)

def cosine_similarity(v1, v2):
    return F.cosine_similarity(v1.unsqueeze(0), v2.unsqueeze(0)).item()
 
new_image_feature = get_feature_for_new_image(new_image_path)


GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
PINECONE_API_KEY = os.getenv("PINE_CONE_API_KEY")

index_name = "image-clip-index"
pinecone = Pinecone(api_key=PINECONE_API_KEY)

existing_indexes = [i["name"] for i in pinecone.list_indexes()]
if index_name not in  existing_indexes:
    pinecone.create_index(
        name=index_name,
        dimension=512,   
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}'aleardy exist.")   
    
     
index = pinecone.Index(index_name) 



def image_embed_store(image_paths):
     
    vectors_to_upsert = []
    images = [preprocess(Image.open(image_path).convert("RGB")) for image_path in image_paths]
    image_input = torch.stack(images).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input).float()  
    for idx, feature in enumerate(image_features): 
        image_filename = os.path.basename(image_paths[idx])          
        vector_id = os.path.splitext(image_filename)[0]      
        numeric_id = vector_id.split("_")[-1] 
        vector_data = feature.cpu().tolist() 
        metadata={"source": "armanstore.com"}
        vectors_to_upsert.append((numeric_id, vector_data, metadata))

    index.upsert(vectors=vectors_to_upsert)
    print("Image features stored in Pinecone.")

    
if index_name not in existing_indexes:
    image_embed_store(image_paths)
    
query_vector = new_image_feature.cpu().tolist()

results = index.query(vector=query_vector, top_k=5)
print(results)
for match in results['matches']:
    print(f"Matched ID: {match['id']}, with score: {match['score']:.8f}")


